In [1]:
# Useful tags
# headings<h1></h1> ....<h6></h6>
# paragraph <p></p>
# line break<br>
# link with attribute
# <a href = "http://www.example.com/">An example link</a>

# Web scraping in Python (Part 1): Getting started

In [36]:
# HTML Consists of Tags
# Tags can have attributes
# Tags can be nested

In [2]:
import requests
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')

In [3]:
# print the first 500 characters of the HTML
print(r.text[0:500])

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--><html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope xmlns:og="http://opengraphprotocol.org/schema/"><!--<![endif]-->
<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 page-interactive section-opinion page


# Parsing the HTML using Beautiful Soup

In [6]:

from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

# Collecting all of the records

 
 <span class = "short-desc"><strong> DATE </strong> LIE <span class = "short-truth"> <a herf = "URL"> EXPLANATION </a></span></span>

In [42]:
results = soup.find_all('span', attrs = {'class': 'short-desc'})

In [43]:
len(results)

180

In [46]:
results[0:3]

[<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>,
 <span class="short-desc"><strong>Jan. 21 </strong>“A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.” <span class="short-truth"><a href="http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/" target="_blank">(Trump was on the cover 11 times and Nixon appeared 55 times.)</a></span></span>,
 <span class="short-desc"><strong>Jan. 23 </strong>“Between 3 million and 5 million illegal votes caused me to lose the popular vote.” <span class="short-truth"><a href="https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html" target="_

In [47]:
results[-1]

<span class="short-desc"><strong>Nov. 11 </strong>“I'd rather have him  – you know, work with him on the Ukraine than standing and arguing about whether or not  – because that whole thing was set up by the Democrats.” <span class="short-truth"><a href="https://www.nytimes.com/interactive/2017/12/10/us/politics/trump-and-russia.html" target="_blank">(There is no evidence that Democrats "set up" Russian interference in the election.)</a></span></span>

# Web scraping in Python (Part 3): Building a dataset

# Extracting the date

In [48]:
first_result = results[0]
first_result

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

In [49]:
first_result.find('strong')
# Results a tag

<strong>Jan. 21 </strong>

In [51]:
first_result.find('strong').text
# Returns a text in a tag

'Jan. 21\xa0'

In [52]:
first_result.find('strong').text[0:-1]
# To chop-off \xa0 from the end

'Jan. 21'

In [54]:
first_result.find('strong').text[0:-1] + ',2017'
# Add year to avoid ambiguity in the date

'Jan. 21,2017'

# Extract the Lie

In [55]:
first_result

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

In [58]:
first_result.contents
# For tags and strings nested in a tag

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

In [61]:
first_result.contents[1]
# Second element of nested tags

"“I wasn't a fan of Iraq. I didn't want to go into Iraq.” "

In [63]:
first_result.contents[1][1:-2]
# To chop off the quotes at the end

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

# Extracting the Explanation

In [64]:
first_result.contents[2]

<span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>

In [65]:
first_result.find('a')
# To search for the surrounding tag

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

In [67]:
first_result.find('a').text[1:-1]
# To only extract the sentence

'He was for an invasion before he was against it.'

# Extracting the URL

In [68]:
first_result.find('a')

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

In [70]:
first_result.find('a')['href']
# Extract the href attribute between the a tag

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

In [77]:
first_result.text

"Jan. 21\xa0“I wasn't a fan of Iraq. I didn't want to go into Iraq.” (He was for an invasion before he was against it.)"

# Building the dataset with loop

In [78]:
records = []

In [89]:
records = []
for result in results:
    date = result.find('strong').text[0:-1] + ',2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date,lie,explanation,url))

In [90]:
len(records)

180

In [91]:
records[0:3]

[('Jan. 21,2017',
  "I wasn't a fan of Iraq. I didn't want to go into Iraq.",
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 21,2017',
  'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.',
  'Trump was on the cover 11 times and Nixon appeared 55 times.',
  'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/'),
 ('Jan. 23,2017',
  'Between 3 million and 5 million illegal votes caused me to lose the popular vote.',
  "There's no evidence of illegal voting.",
  'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html')]

# Web scraping in Python (Part 4): Exporting a CSV with pandas

# Applying a tabular data structure

In [93]:
records = []
for result in results:
    date = result.find('strong').text[0:-1] + ',2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date,lie,explanation,url))

In [98]:
import pandas as pd
df = pd.DataFrame(records, columns = ['date','lie','explanation','url'])

In [99]:
df.head()

,date,lie,explanation,url
0,"Jan. 21,2017",I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,"Jan. 21,2017",A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,"Jan. 23,2017",Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,"Jan. 25,2017","Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,"Jan. 25,2017",Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...


In [101]:
df.tail()

,date,lie,explanation,url
175,"Oct. 25,2017",We have trade deficits with almost everybody.,We have trade surpluses with more than 100 cou...,https://www.bea.gov/newsreleases/international...
176,"Oct. 27,2017","Wacky & totally unhinged Tom Steyer, who has b...",Steyer has financially supported many winning ...,https://www.opensecrets.org/donor-lookup/resul...
177,"Nov. 1,2017","Again, we're the highest-taxed nation, just ab...",We're not.,http://www.politifact.com/truth-o-meter/statem...
178,"Nov. 7,2017",When you look at the city with the strongest g...,"Several other cities, including New York and L...",http://www.politifact.com/truth-o-meter/statem...
179,"Nov. 11,2017","I'd rather have him – you know, work with him...","There is no evidence that Democrats ""set up"" R...",https://www.nytimes.com/interactive/2017/12/10...


# Export the dataset to a CSV File

In [105]:
df.to_csv('trump_lies.csv',index = False, encoding = 'utf-8')

In [107]:
df = pd.read_csv('trump_lies.csv', parse_dates = ['date'],encoding = 'utf-8')